In [2]:
import pickle
import pandas as pd
import numpy as np
from drfp import DrfpEncoder
from functools import partial
from typing import Iterable
import multiprocessing
import os

[12:10:28] Enabling RDKit 2019.09.3 jupyter extensions
RDKit WARNING: [12:10:28] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
# Define current working directory so we can navigate folders easily
cwd = os.getcwd()
print(cwd)
os.chdir('../')
pwd = os.getcwd()
print(pwd)

/home/gah/oprd/jupyter
/home/gah/oprd


In [61]:
# Read in our dataset with added reagents
data = pd.read_csv(pwd+'/data/reagents_added.csv')

In [73]:
# For now, only deal with reactions which have a single product - purely for plotting reasons
data1 = data[~data['Yield (numerical)'].str.contains(';')]

In [74]:
# Some reactions have more than one time or temperature
# I am going to take the first time/temperature as these are most 
# representative of the reactivity of the starting materials

temps_split = [i.split("; ") for i in data1['Temperature (Reaction Details) [C]'].to_list()]
# Take first temp if there is more than one
temps_initial = [i[0] for i in temps_split]
# Split upper and lower bound if range present
temps_ranges = [i.split(' - ') for i in temps_initial]
# Make a new list which stores the average of the range as new temp
new_temps = []
for i in temps_ranges:
    if len(i) == 1:
        new_temps.append(float(i[0]))
    else:
        new_temps.append((float(i[0])+float(i[1]))/2)

In [75]:
# In the same fashion I will take the first reaction time reported
times_split = [i.split("; ") for i in data1['Time (Reaction Details) [h]'].to_list()]
# Take first temp if there is more than one
times_initial = [i[0] for i in times_split]


### The functions in the code box below are taken from the DRFP paper.
url  ="http://dx.doi.org/10.1039/D1DD00006C"

In [6]:
def encode(smiles: Iterable, length: int = 2048, radius: int = 3) -> np.ndarray: 
    return DrfpEncoder.encode(
        smiles,
        n_folded_length=length,
        radius=radius,
        rings=True,
    )


def encode_dataset(smiles: Iterable, length: int, radius: int) -> np.ndarray:
    """Encode the reaction SMILES to drfp"""

    cpu_count = (
        multiprocessing.cpu_count()
    )  # Data gets too big for piping when splitting less in python < 2.8

    # Split reaction SMILES for multiprocessing
    k, m = divmod(len(smiles), cpu_count)
    smiles_chunks = (
        smiles[i * k + min(i, m) : (i + 1) * k + min(i + 1, m)]
        for i in range(cpu_count)
    )

    # Run the fingerprint generation in parallel
    results = []
    with multiprocessing.Pool(cpu_count) as p:
        results = p.map(partial(encode, length=length, radius=radius), smiles_chunks)

    return np.array([item for s in results for item in s])

In [77]:
# Encode the reactions with Reagents added
drfps = encode_dataset(data1['Reactants.Reagents>>Products'], length=2048, radius=3)

In [85]:
# Encode the reactions without Reagents added for later comparison of classification system
drfp_without_reagents = encode_dataset(data1['Reaction'], length=2048, radius=3)

In [ ]:
# Gather temps, times and yields to be stored with drfps
# Get reaction SMILES too for visualisation with TMAP
yields = data1['Yield (numerical)'].astype(float).to_numpy()
times = np.array(times_initial)
temps = np.array(new_temps)
SMILES = data1['Reactants.Reagents>>Products'].to_numpy()

In [ ]:
# Store the drfps with the contextual data in pickle file
with open(pwd+"/data/oprd_drfp_with_reagents.pkl", "wb+") as f:
        pickle.dump((drfps, yields, times, temps, SMILES), f, protocol=4)

In [ ]:
# Do the same as above for drfps without reagents
with open(pwd+"/data/oprd_drfp_without_reagents.pkl", "wb+") as f:
        pickle.dump((drfp_without_reagents, yields, times, temps, SMILES), f, protocol=4)

In [78]:
# An example of what bits are set to be "on"
drfps[4000].nonzero()

(array([  14,   35,   45,   56,   99,  151,  155,  166,  180,  199,  270,
         319,  344,  392,  418,  435,  444,  466,  475,  483,  505,  594,
         646,  659,  684,  800,  947, 1023, 1053, 1061, 1066, 1089, 1142,
        1291, 1406, 1439, 1491, 1500, 1524, 1544, 1561, 1603, 1607, 1626,
        1630, 1879, 1894]),)

In [79]:
# For the corresponding reaction
data1['Reactants.Reagents>>Products'][4000]

'FC1=CC=C(C=C1)C#N.C[C@H]1CNC[C@@H](C)N1.[Ca++].CCC([O-])=O.CCC([O-])=O>>C[C@H]1CN(C[C@@H](C)N1)C1=CC=C(C=C1)C#N'

## Encoding of query reactions for use in analysis of ANN

From the Reactions with class notebook, import the selected random reactions from the 5 chosen reaction classes and compute the DRFP for each of them, store them with their reaction class and SMILES. These will be used as queries to the LSH forest index of the OPRD reactions to determine the quality of the classification.

In [11]:
# Load in the data from the Compute_DRFP.ipynb notebook
query_reactions = pickle.load(open(pwd+'/data/query_reactions.pkl',"rb"))
query_SMILES = query_reactions[0]
reaction_types = query_reactions[1]

In [12]:
# Encode the query reactions
query_drfp = encode_dataset(query_SMILES, 2048, 3)

In [14]:
# Save it as a pickle file
with open(pwd+"/data/query_reactions_drfp.pkl", "wb+") as f:
        pickle.dump((query_drfp, query_SMILES ,reaction_types), f, protocol=4)